In [ ]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import numpy as np
import pandas as pd

In [ ]:
setup_logging()

In [ ]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only = True)

In [ ]:
datasets = Dataset.get_all_dataset_names()
print(len(datasets)) #Total number of datasets obtained using the API

In [ ]:
#cleaning all the selected datasets
def clean(datasets, file_types=[]):
    result = []
    index = 0
    while (index < len(datasets)):
        temp_dataset = Dataset.read_from_hdx(datasets[index])
        temp_dataset.separate_resources() #In order to use dataset.resources
        if (len(temp_dataset.resources) > 0):
            if (len(file_types) > 0):
                if (not set(temp_dataset.get_filetypes()).isdisjoint(file_types)): #To check if elements of the 2 lists of file_types overlap.
                    result.append(datasets[index])
            else :
                result.append(datasets[index])
        index += 1
    return result

In [ ]:
def dload_dset(dataset_1,dload_path,row_limit=10,file_type='CSV') :
    if (file_type == None):
        url, path = dataset_1.resources[0].download(dload_path)
        pandas_dataset = pd.read_csv(path)
    else :
        if (file_type not in dataset_1.get_filetypes()):
            url = '1'
            path = '2'
            pandas_dataset ='9'
            return 'Error: Required file type not in dataset OR dataset does not contain any resources.'
        else :
            url, path = dataset_1.resources[dataset_1.get_filetypes().index(file_type)].download(dload_path)
            try:
                pandas_dataset = pd.read_csv(path, encoding='latin-1')
                os.remove(path)
                pandas_dataset = pandas_dataset.head(row_limit)
            except:
                pandas_dataset = pd.DataFrame()
        return pandas_dataset

In [ ]:
def hdx_dset_scraper(dload_path,limit=1000000000,download=True,file_type='CSV'):
    datasets = Dataset.get_all_dataset_names()
    if (limit > len(datasets)):
        limit = len(datasets)
    dset_dict = {}
    index = 0
    while (limit > index) :
        dataset_1 = Dataset.read_from_hdx(datasets[index]) 
        dataset_1.separate_resources()
        if (file_type not in dataset_1.get_filetypes()) :
            index += 1
            limit += 1
        else :
            dset_dict.update({datasets[index] : dload_dset(dataset_1,dload_path)})
            index += 1
    return dset_dict

In [ ]:
dict_10 = hdx_dset_scraper('Datasets',limit=50)

In [ ]:
#storing all data into a dataframe
df = pd.DataFrame.from_dict(data = dict_10, orient = 'index')

#storing all the column names
list(df.iloc[0,0].columns.values)
col_names = [list(df.iloc[i,0].columns.values) for i in range(len(df))]